# Análise Estatística: SmartRefactor vs. Refatoração Tradicional

Este notebook reproduz, em células interativas, todo o fluxo de análise realizado no script Python:
- Carregamento e limpeza dos dados
- Estatísticas descritivas
- Testes de hipótese
- Análise de perfil dos participantes
- Geração de gráficos (boxplots)

As chamadas abaixo seguem exatamente a lógica e o código final do script fornecido.

## 1. Configuração de Ambiente e Importações

In [2]:
from pathlib import Path
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import math

# Caminhos para os arquivos e diretório de saída
metrics_csv = Path("./coleta.csv")
profile_csv = Path("./perfil_dos_participantes.csv")
out_dir = Path("./results")
out_dir.mkdir(parents=True, exist_ok=True)

## 2. Função `load_metrics`
Leitura e limpeza dos dados de métricas.

In [3]:
def load_metrics(csv_path: Path) -> pd.DataFrame:
    df_raw = pd.read_csv(csv_path)
    rename = {
        "Tempo (h)": "tempo_h",
        "LOC Modificadas": "loc_mod",
        "Erros Funcionais": "erros",
        "Problemas de Design": "design",
        "Ferramenta": "ferramenta",
    }
    df = df_raw.rename(columns=rename)
    for col in ["tempo_h", "loc_mod"]:
        df[col] = df[col].astype(str).str.strip()\
                                 .str.replace(",", ".", regex=False)\
                                 .astype(float)
    df["erros"] = pd.to_numeric(df["erros"], errors="coerce")
    df["design"] = pd.to_numeric(df["design"], errors="coerce")
    df = df[["ID", "tempo_h", "erros", "design", "ferramenta"]]
    return df

# Carregar e exibir as cinco primeiras linhas
df_metrics = load_metrics(metrics_csv)
df_metrics.head()

,ID,tempo_h,erros,design,ferramenta
0,P1,2.600586,3,3,Tradicional
1,P2,2.222081,2,5,Tradicional
2,P3,2.855597,2,5,Tradicional
3,P4,2.964427,0,3,Tradicional
4,P5,2.460526,5,6,Tradicional


## 3. Função `load_profile`
Leitura dos dados de perfil dos participantes.

In [4]:
def load_profile(csv_path: Path) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    rename = {
        "Formacao": "formacao",
        "Experiencia": "experiencia",
        "Conhecimento_Refatoracao": "kn_ref",
        "Conhecimento_Java": "kn_java"
    }
    return df.rename(columns=rename)

df_profile = load_profile(profile_csv)
df_profile.head()

,ID,formacao,experiencia,kn_ref,kn_java
0,P1,Mestrado,6+ anos,Nenhum,Razoável
1,P2,Graduação,1-2 anos,Razoável,Básico
2,P3,Graduação,1-2 anos,Avançado,Razoável
3,P4,Mestrado,3-5 anos,Básico,Básico
4,P5,Mestrado,6+ anos,Nenhum,Básico


## 4. União dos DataFrames
Merge por `ID` para relacionar perfil e métricas.

In [5]:
df = pd.merge(df_metrics, df_profile, on="ID", how="left")
df.head()

,ID,tempo_h,erros,design,ferramenta,formacao,experiencia,kn_ref,kn_java
0,P1,2.600586,3,3,Tradicional,Mestrado,6+ anos,Nenhum,Razoável
1,P2,2.222081,2,5,Tradicional,Graduação,1-2 anos,Razoável,Básico
2,P3,2.855597,2,5,Tradicional,Graduação,1-2 anos,Avançado,Razoável
3,P4,2.964427,0,3,Tradicional,Mestrado,3-5 anos,Básico,Básico
4,P5,2.460526,5,6,Tradicional,Mestrado,6+ anos,Nenhum,Básico


## 5. Estatística Descritiva por Ferramenta

In [6]:
def descriptive_stats(df: pd.DataFrame) -> pd.DataFrame:
    return (df.groupby("ferramenta")
              .agg(n=("tempo_h","count"),
                   tempo_medio=("tempo_h","mean"),
                   tempo_dp=("tempo_h","std"),
                   erros_medio=("erros","mean"),
                   erros_dp=("erros","std"),
                   design_medio=("design","mean"),
                   design_dp=("design","std"))
              .round(3))

desc = descriptive_stats(df)
desc.to_csv(out_dir/"descriptive_stats.csv")
desc

,n,tempo_medio,tempo_dp,erros_medio,erros_dp,design_medio,design_dp
ferramenta,,,,,,,
SmartRefactor,60,2.043,0.367,0.867,0.853,2.900,1.664
Tradicional,60,2.480,0.386,2.317,1.396,4.717,2.344


## 6. Testes de Hipótese

In [7]:
def hypothesis_tests(df: pd.DataFrame) -> pd.DataFrame:
    results = []
    for metric, label in [
        ("tempo_h","Tempo (h)"),
        ("erros","Erros Funcionais"),
        ("design","Problemas de Design")]:
        grp1 = df[df["ferramenta"]=="SmartRefactor"][metric]
        grp2 = df[df["ferramenta"]=="Tradicional"][metric]
        p1 = stats.shapiro(grp1).pvalue
        p2 = stats.shapiro(grp2).pvalue
        normal = (p1>0.05 and p2>0.05)
        if normal:
            stat,pval = stats.ttest_ind(grp1,grp2,equal_var=False); test="t (Welch)"
            effect = abs(grp1.mean()-grp2.mean())/math.sqrt((grp1.var(ddof=1)+grp2.var(ddof=1))/2)
        else:
            stat,pval = stats.mannwhitneyu(grp1,grp2,alternative="two-sided"); test="Mann–Whitney"
            z = stats.norm.isf(pval/2); effect = abs(z)/(len(grp1)+len(grp2))**0.5
        results.append({"Métrica":label,"Teste":test,
                        "Estatística":round(stat,3),
                        "p_valor":round(pval,5),
                        "Normal?":normal,
                        "Efeito":round(effect,3)})
    return pd.DataFrame(results)

hyp = hypothesis_tests(df)
hyp.to_csv(out_dir/"hypothesis_results.csv", index=False)
hyp

,Métrica,Teste,Estatística,p_valor,Normal?,Efeito
0,Tempo (h),t (Welch),-6.357,0.00000,True,1.161
1,Erros Funcionais,Mann–Whitney,697.000,0.00000,False,0.543
2,Problemas de Design,Mann–Whitney,986.000,0.00002,False,0.394


## 7. Estatísticas do Perfil dos Participantes

In [8]:
def profile_descriptive(df_profile: pd.DataFrame) -> pd.DataFrame:
    counts = {col: df_profile[col].value_counts() for col in df_profile.columns if col!='ID'}
    return pd.DataFrame(counts)

pdesc = profile_descriptive(df_profile)
pdesc.to_csv(out_dir/"profile_stats.csv")
pdesc

,formacao,experiencia,kn_ref,kn_java
1-2 anos,NaN,29.0,NaN,NaN
3-5 anos,NaN,51.0,NaN,NaN
6+ anos,NaN,25.0,NaN,NaN
< 1 ano,NaN,15.0,NaN,NaN
Avançado,NaN,NaN,27.0,23.0
Básico,NaN,NaN,41.0,32.0
Especialização,17.0,NaN,NaN,NaN
Graduação,58.0,NaN,NaN,NaN
Mestrado,45.0,NaN,NaN,NaN
Nenhum,NaN,NaN,15.0,9.0


## 8. Influência do Perfil sobre Métricas

In [9]:
def map_ordinal(series, order):
    return series.map({cat:i for i,cat in enumerate(order)}).astype(float)

df["exp_num"] = map_ordinal(df["experiencia"],["0-1 anos","1-2 anos","3-5 anos","6+ anos"])
df["kn_ref_num"] = map_ordinal(df["kn_ref"],["Nenhum","Básico","Razoável","Avançado"])
df["kn_java_num"] = map_ordinal(df["kn_java"],["Nenhum","Básico","Razoável","Avançado"])

def profile_influence(df):
    res=[]
    for col,label in [("exp_num","Experiência"),("kn_ref_num","KN Ref"),("kn_java_num","KN Java")]:
        for metric in ["erros","design"]:
            coef,p = stats.spearmanr(df[col],df[metric],nan_policy="omit")
            res.append((label, metric, "Spearman", round(coef,3), round(p,5)))
    # Kruskal
    groups = [df[df["experiencia"]==lvl]["erros"] for lvl in ["0-1 anos","1-2 anos","3-5 anos","6+ anos"]]
    stat,p = stats.kruskal(*groups)
    res.append(("Experiência (cat)","Erros","Kruskal",round(stat,3),round(p,5)))
    return pd.DataFrame(res, columns=["Variável","Métrica","Teste","Estat","p_valor"])

pinf = profile_influence(df)
pinf.to_csv(out_dir/"profile_influence.csv", index=False)
pinf

/var/folders/m7/0vkmn80144lcqcp11ggbb3v00000gn/T/ipykernel_22521/2101752209.py:16: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  stat,p = stats.kruskal(*groups)


,Variável,Métrica,Teste,Estat,p_valor
0,Experiência,erros,Spearman,0.083,0.40001
1,Experiência,design,Spearman,-0.058,0.55912
2,KN Ref,erros,Spearman,0.141,0.12349
3,KN Ref,design,Spearman,0.027,0.77213
4,KN Java,erros,Spearman,-0.043,0.63995
5,KN Java,design,Spearman,-0.052,0.57438
6,Experiência (cat),Erros,Kruskal,NaN,NaN


## 9. Box-plots das Métricas

In [10]:
for metric,label in [("tempo_h","Tempo (h)"),("erros","Erros"),("design","Design")]:
    plt.figure()
    df.boxplot(column=metric,by="ferramenta")
    plt.title(label+" por ferramenta")
    plt.suptitle("")
    plt.savefig(out_dir/f"boxplot_{metric}.png",dpi=300)
    plt.close()

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>